# Load File and get metadata
Used this file for prototyping

In [2]:
from pathlib import Path
from music21 import chord, stream
import music21
import streamlit as st
import cv2
import shutil
xml_path = "megalovania.mxl"

score = music21.converter.parse(xml_path)
total_measures = len(score.parts[0].getElementsByClass(stream.Measure))
print(score)


<music21.stream.Score 0x151f80760>


In [ ]:
skip = False
png_folder = Path(xml_path).parent / Path(xml_path).stem / "png"
png_paths = []
png_folder.mkdir(parents=True, exist_ok=True)
skips = []
for i in range(1, total_measures):
    if not skip:
        print(f"Converting measure {i} to png")
        measures_stream = score.measure(i)

        my_stream = stream.Stream()
        for stream_ in measures_stream:
            my_stream.append(stream_)
        output_path = my_stream.write("musicxml.png", Path("temp.png"))
        #monkey patch: 
        # if file size < 1mb, its a page break, skip it
        if Path(output_path).stat().st_size < 1024:
            print(f"Skipping page break{i}")
            skips.append(i)
            continue
    png_path = png_folder / f"{i}.png"
    png_paths.append(png_path)
    if not skip:
        shutil.move(output_path, png_path)
print(skips)




Converting measure 1 to png
[]
Converting measure 2 to png
[]
Converting measure 3 to png
[]
Converting measure 4 to png
[]
Converting measure 5 to png
[]
Converting measure 6 to png
[]
Converting measure 7 to png
[]
Converting measure 8 to png
[]
Converting measure 9 to png
[]
Converting measure 10 to png
[]
Converting measure 11 to png
Skipping page break11
Converting measure 12 to png
[11]
Converting measure 13 to png
[11]
Converting measure 14 to png
[11]
Converting measure 15 to png
[11]
Converting measure 16 to png
[11]
Converting measure 17 to png
[11]
Converting measure 18 to png
[11]
Converting measure 19 to png
[11]
Converting measure 20 to png
[11]
Converting measure 21 to png
Skipping page break21
Converting measure 22 to png
[11, 21]
Converting measure 23 to png
[11, 21]
Converting measure 24 to png
[11, 21]
Converting measure 25 to png
[11, 21]
Converting measure 26 to png
[11, 21]
Converting measure 27 to png
[11, 21]
Converting measure 28 to png
[11, 21]
Converting meas

In [ ]:
# slice the pngs to the highest row with a black pixel
# cropped pngs folder
import cv2

cropped_folder = Path(xml_path).parent / Path(xml_path).stem / "cropped_png"
cropped_folder.mkdir(parents=True, exist_ok=True)
cropped_png_paths = []

def is_black(pixel):
    return pixel < 10  # Adjust the threshold as needed

for png_file in png_paths:
    print(f"Cropping {png_file}")
    img = cv2.imread(str(png_file), cv2.IMREAD_UNCHANGED)
    # fix transparency -> white background
    # https://stackoverflow.com/questions/31656366/cv2-imread-and-cv2-imshow-return-all-zeros-and-black-image
    if img.shape[2] == 4:     # we have an alpha channel
        a1 = ~img[:,:,3]        # extract and invert that alpha
        img = cv2.add(cv2.merge([a1,a1,a1,a1]), img)   # add up values (with clipping)
        img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    top_row_index = 0
    while top_row_index < len(gray) and not any(is_black(pixel) for pixel in gray[top_row_index]):
        top_row_index += 1
    
    bottom_row_index = len(gray) - 1
    while bottom_row_index > top_row_index and not any(is_black(pixel) for pixel in gray[bottom_row_index]):
        bottom_row_index -= 1
    
    cropped_img = img[top_row_index:bottom_row_index+1]
    cropped_path = cropped_folder / png_file.name
    cropped_png_paths.append(cropped_path)
    cv2.imwrite(str(cropped_path), cropped_img)
    print(f"Saved cropped image to {cropped_path}")



Cropping megalovania/png/1.png
Saved cropped image to megalovania/cropped_png/1.png
Cropping megalovania/png/2.png
Saved cropped image to megalovania/cropped_png/2.png
Cropping megalovania/png/3.png
Saved cropped image to megalovania/cropped_png/3.png
Cropping megalovania/png/4.png
Saved cropped image to megalovania/cropped_png/4.png
Cropping megalovania/png/5.png
Saved cropped image to megalovania/cropped_png/5.png
Cropping megalovania/png/6.png
Saved cropped image to megalovania/cropped_png/6.png
Cropping megalovania/png/7.png
Saved cropped image to megalovania/cropped_png/7.png
Cropping megalovania/png/8.png
Saved cropped image to megalovania/cropped_png/8.png
Cropping megalovania/png/9.png
Saved cropped image to megalovania/cropped_png/9.png
Cropping megalovania/png/10.png
Saved cropped image to megalovania/cropped_png/10.png
Cropping megalovania/png/12.png
Saved cropped image to megalovania/cropped_png/12.png
Cropping megalovania/png/13.png
Saved cropped image to megalovania/cropp

In [18]:
# convert cropped pngs to bitmaps
bitmap_folder = Path(xml_path).parent / Path(xml_path).stem / "bitmap"
bitmap_folder.mkdir(parents=True, exist_ok=True)
bitmap_paths = []
for cropped_png_path in cropped_png_paths:
    print(f"Converting {cropped_png_path} to bitmap")
    img = cv2.imread(str(cropped_png_path), cv2.IMREAD_GRAYSCALE)
    ret, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    bitmap_path = str(bitmap_folder / cropped_png_path.stem) + ".bmp"
    bitmap_paths.append(bitmap_path)
    cv2.imwrite(str(bitmap_path), img)
    print(f"Saved bitmap to {bitmap_path}")

Converting megalovania/cropped_png/1.png to bitmap
Saved bitmap to megalovania/bitmap/1.bmp
Converting megalovania/cropped_png/2.png to bitmap
Saved bitmap to megalovania/bitmap/2.bmp
Converting megalovania/cropped_png/3.png to bitmap
Saved bitmap to megalovania/bitmap/3.bmp
Converting megalovania/cropped_png/4.png to bitmap
Saved bitmap to megalovania/bitmap/4.bmp
Converting megalovania/cropped_png/5.png to bitmap
Saved bitmap to megalovania/bitmap/5.bmp
Converting megalovania/cropped_png/6.png to bitmap
Saved bitmap to megalovania/bitmap/6.bmp
Converting megalovania/cropped_png/7.png to bitmap
Saved bitmap to megalovania/bitmap/7.bmp
Converting megalovania/cropped_png/8.png to bitmap
Saved bitmap to megalovania/bitmap/8.bmp
Converting megalovania/cropped_png/9.png to bitmap
Saved bitmap to megalovania/bitmap/9.bmp
Converting megalovania/cropped_png/10.png to bitmap
Saved bitmap to megalovania/bitmap/10.bmp
Converting megalovania/cropped_png/12.png to bitmap
Saved bitmap to megalovani